In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import time
import numpy
from pandas import *
from tkinter import *

File parser

In [3]:
"""
4 5     # number of rows and cols
A 4 0   # agent starting coordinates
W 1 0   # wumpus coordinates
G 1 1   # gold coordinates
P 0 3   # 1st pit coordinates
P 1 2   # 2nd pit coordinates
P 3 2   # 3rd pit coordinates
"""

class File_Parser:
    def __init__(self, world_file):
        self.row_col = []
        self.agent = []
        self.wumpus = []
        self.gold = []
        self.pits = [[]]
        file = open(world_file, 'r')
        self.row_col = file.readline()
        self.row_col = self.row_col.rstrip('\r\n')
        self.row_col = self.row_col.split(" ")
        # print(self.row_col)

        self.agent = file.readline()
        self.agent = self.agent.rstrip('\r\n')
        self.agent = self.agent.split(" ")
        self.wumpus = file.readline()
        self.wumpus = self.wumpus.rstrip('\r\n')
        self.wumpus = self.wumpus.split(" ")
        self.gold = file.readline()
        self.gold = self.gold.rstrip('\r\n')
        self.gold = self.gold.split(" ")
        self.pits = []
        while True:
            pit = file.readline()
            if len(pit) == 0:
                break
            pit = pit.rstrip('\r\n')
            pit = pit.split(" ")
            self.pits.append(pit)



World

In [4]:
class World:
    def __init__(self):
        self.world = [[]]
        self.num_rows = 0
        self.num_cols = 0
        self.agent_row = 0
        self.agent_col = 0
        self.cave_entrance_row = 0
        self.cave_entrance_col = 0

    def generate_world(self, file_name):
        file_parser = File_Parser(file_name)
        self.num_rows = int(file_parser.row_col[0])
        self.num_cols = int(file_parser.row_col[1])
        self.world = [[[] for i in range(self.num_cols)] for j in range(self.num_rows)]
        self.agent_row = int(file_parser.agent[1])
        self.agent_col = int(file_parser.agent[2])
        self.world[self.agent_row][self.agent_col].append('A')
        self.world[int(file_parser.wumpus[1])][int(file_parser.wumpus[2])].append(file_parser.wumpus[0])
        self.world[int(file_parser.gold[1])][int(file_parser.gold[2])].append(file_parser.gold[0])
        for pit in file_parser.pits:
            self.world[int(pit[1])][int(pit[2])].append(pit[0])
        self.populate_indicators()

    def populate_indicators(self):
        for i in range(self.num_rows):
            for j in range(self.num_cols):
                for k in range(len(self.world[i][j])):
                    if self.world[i][j][k] == 'W':
                        try:
                            if i-1 >= 0:
                                if 'S' not in self.world[i-1][j]:
                                    self.world[i-1][j].append('S')
                        except IndexError:
                            pass
                        try:
                            if j+1 < self.num_cols:
                                if 'S' not in self.world[i][j+1]:
                                    self.world[i][j+1].append('S')
                        except IndexError:
                            pass
                        try:
                            if i+1 < self.num_rows:
                                if 'S' not in self.world[i+1][j]:
                                    self.world[i+1][j].append('S')
                        except IndexError:
                            pass
                        try:
                            if j-1 >= 0:
                                if 'S' not in self.world[i][j-1]:
                                    self.world[i][j-1].append('S')
                        except IndexError:
                            pass

                    if self.world[i][j][k] == 'P':
                        try:
                            if i-1 >= 0:
                                if 'B' not in self.world[i-1][j]:
                                    self.world[i-1][j].append('B')
                        except IndexError:
                            pass
                        try:
                            if j+1 < self.num_cols:
                                if 'B' not in self.world[i][j+1]:
                                    self.world[i][j+1].append('B')
                        except IndexError:
                            pass
                        try:
                            if i+1 < self.num_rows:
                                if 'B' not in self.world[i+1][j]:
                                    self.world[i+1][j].append('B')
                        except IndexError:
                            pass
                        try:
                            if j-1 >= 0:
                                if 'B' not in self.world[i][j-1]:
                                    self.world[i][j-1].append('B')
                        except IndexError:
                            pass

Agent

In [5]:
"""
Legend:
. = visited tile
A = agent
G = gold
W = wumpus
S = stench
w = potential wumpus
nw = no wumpus
P = pit
B = breeze
p = potential pit
np = no pit
"""
class Agent:
    def __init__(self, world, label_grid):
        self.world = world
        self.world_knowledge = [[[] for i in range(self.world.num_cols)] for j in range(self.world.num_rows)]
        self.world_knowledge[self.world.agent_row][self.world.agent_col].append('A')
        self.num_stenches = 0
        self.path_out_of_cave = [[self.world.agent_row, self.world.agent_col]]
        self.mark_tile_visited()
        self.world.cave_entrance_row = self.world.agent_row
        self.world.cave_entrance_col = self.world.agent_col
        self.found_gold = False # self.exit_cave(found_gold)
        self.took_gold = False
        self.exited = False
        self.label_grid = label_grid
        self.repaint_world()

    def repaint_world(self):
        for i in range(self.world.num_rows):
            for j in range(self.world.num_cols):
                updated_text = []
                if 'A' in self.world_knowledge[i][j]:
                    updated_text.append('A')
                if 'w' in self.world_knowledge[i][j]:
                    updated_text.append('W')
                if 'p' in self.world_knowledge[i][j]:
                    updated_text.append('P')
                if 'B' in self.world_knowledge[i][j]:
                    updated_text.append('B')
                if 'S' in self.world_knowledge[i][j]:
                    updated_text.append('S')
                if 'G' in self.world_knowledge[i][j]:
                    updated_text.append('G')
                updated_str=""
                self.label_grid[i][j].change_text(updated_str.join(updated_text))
                if '.' in self.world_knowledge[i][j]:
                    self.label_grid[i][j].label.config(bg="gray40")
                self.label_grid[i][j].label.update()

    def go_back_one_tile(self):
        if self.world.agent_row-1 == self.path_out_of_cave[-1][0]:
            self.move('u')
        if self.world.agent_row+1 == self.path_out_of_cave[-1][0]:
            self.move('d')
        if self.world.agent_col+1 ==  self.path_out_of_cave[-1][1]:
            self.move('r')
        if self.world.agent_col-1 ==  self.path_out_of_cave[-1][1]:
            self.move('l')
        del self.path_out_of_cave[-1]

    def leave_cave(self):
        for tile in reversed(self.path_out_of_cave):
            if self.world.agent_row-1 == tile[0]:
                self.move('u')
            if self.world.agent_row+1 == tile[0]:
                self.move('d')
            if self.world.agent_col+1 == tile[1]:
                self.move('r')
            if self.world.agent_col-1 == tile[1]:
                self.move('l')
            if self.world.cave_entrance_row == self.world.agent_row:
                if self.world.cave_entrance_col == self.world.agent_col:
                    if self.found_gold == True:
                        self.exited = True
                        break

    def explore(self):
        last_move = ''
        already_moved = False
        while self.found_gold == False:
            if self.found_gold == True:
                break
            try:
                if  '.' not in self.world_knowledge[self.world.agent_row-1][self.world.agent_col] and self.is_safe_move(self.world.agent_row-1, self.world.agent_col):
                    if already_moved == False:
                        if self.move('u'):
                            already_moved = True
            except IndexError:
                pass
            try:
                if '.' not in self.world_knowledge[self.world.agent_row][self.world.agent_col+1] and self.is_safe_move(self.world.agent_row, self.world.agent_col+1):
                    if already_moved == False:
                        if self.move('r'):
                            already_moved = True
                    # print(self.path_out_of_cave)
            except IndexError:
                pass
            try:
                if '.' not in self.world_knowledge[self.world.agent_row+1][self.world.agent_col] and self.is_safe_move(self.world.agent_row+1, self.world.agent_col):
                    if already_moved == False:
                        if self.move('d'):
                            already_moved = True
            except IndexError:
                pass
            try:
                if '.' not in self.world_knowledge[self.world.agent_row][self.world.agent_col-1] and self.is_safe_move(self.world.agent_row, self.world.agent_col-1):
                    if already_moved == False:
                        if self.move('l'):
                            already_moved = True
            except IndexError:
                pass
            if already_moved == False:
                self.go_back_one_tile()
            already_moved = False

    def move(self, direction):
        self.repaint_world()
        if self.found_gold == True and self.took_gold == False:
            self.took_gold == True
            if 'G' in self.world_knowledge[self.world.agent_row][self.world.agent_col]:
                self.world_knowledge[self.world.agent_row][self.world.agent_col].remove('G')
        successful_move = False
        if direction == 'u':
            if self.is_safe_move(self.world.agent_row-1, self.world.agent_col):
                successful_move = self.move_up()
        if direction == 'r':
            if self.is_safe_move(self.world.agent_row, self.world.agent_col+1):
                successful_move = self.move_right()
        if direction == 'd':
            if self.is_safe_move(self.world.agent_row+1, self.world.agent_col):
                successful_move = self.move_down()
        if direction == 'l':
            if self.is_safe_move(self.world.agent_row, self.world.agent_col-1):
                successful_move = self.move_left()
        if successful_move:
            self.add_indicators_to_knowledge()
            self.mark_tile_visited()
            self.predict_wumpus()
            self.predict_pits()
            self.clean_predictions()
            self.confirm_wumpus_knowledge()
            if 'G' in self.world_knowledge[self.world.agent_row][self.world.agent_col]:
                self.found_gold = True
            if self.found_gold == False:
                self.path_out_of_cave.append([self.world.agent_row, self.world.agent_col])
            time.sleep(1.5)
        return successful_move

    def add_indicators_to_knowledge(self):
        if 'B' in self.world.world[self.world.agent_row][self.world.agent_col]:
            if 'B' not in self.world_knowledge[self.world.agent_row][self.world.agent_col]:
                self.world_knowledge[self.world.agent_row][self.world.agent_col].append('B')
        if 'S' in self.world.world[self.world.agent_row][self.world.agent_col]:
            if 'S' not in self.world_knowledge[self.world.agent_row][self.world.agent_col]:
                self.world_knowledge[self.world.agent_row][self.world.agent_col].append('S')
        if 'G' in self.world.world[self.world.agent_row][self.world.agent_col]:
            if 'G' not in self.world_knowledge[self.world.agent_row][self.world.agent_col]:
                self.world_knowledge[self.world.agent_row][self.world.agent_col].append('G')
        if 'P' in self.world.world[self.world.agent_row][self.world.agent_col]:
            if 'P' not in self.world_knowledge[self.world.agent_row][self.world.agent_col]:
                self.world_knowledge[self.world.agent_row][self.world.agent_col].append('P')
        if 'W' in self.world.world[self.world.agent_row][self.world.agent_col]:
            if 'W' not in self.world_knowledge[self.world.agent_row][self.world.agent_col]:
                self.world_knowledge[self.world.agent_row][self.world.agent_col].append('W')

    def predict_pits(self):
        try:
            if 'B' in self.world.world[self.world.agent_row][self.world.agent_col]:
                if self.world.agent_row-1 >= 0:
                    if '.' not in self.world.world[self.world.agent_row-1][self.world.agent_col]:
                        if 'p' not in self.world_knowledge[self.world.agent_row-1][self.world.agent_col]:
                            self.world_knowledge[self.world.agent_row-1][self.world.agent_col].append('p')
        except IndexError:
            pass
        try:
            if 'B' in self.world.world[self.world.agent_row][self.world.agent_col]:
                if self.world.agent_col+1 < self.world.num_cols:
                    if '.' not in self.world.world[self.world.agent_row][self.world.agent_col+1]:
                        if 'p' not in self.world_knowledge[self.world.agent_row][self.world.agent_col+1]:
                            self.world_knowledge[self.world.agent_row][self.world.agent_col+1].append('p')
        except IndexError:
            pass
        try:
            if 'B' in self.world.world[self.world.agent_row][self.world.agent_col]:
                if self.world.agent_row+1 < self.world.num_rows:
                    if '.' not in self.world.world[self.world.agent_row+1][self.world.agent_col]:
                        if 'p' not in self.world_knowledge[self.world.agent_row+1][self.world.agent_col]:
                            self.world_knowledge[self.world.agent_row+1][self.world.agent_col].append('p')
        except IndexError:
            pass
        try:
            if 'B' in self.world.world[self.world.agent_row][self.world.agent_col]:
                if self.world.agent_col-1 >= 0:
                    if '.' not in self.world.world[self.world.agent_row][self.world.agent_col-1]:
                        if 'p' not in self.world_knowledge[self.world.agent_row][self.world.agent_col-1]:
                            self.world_knowledge[self.world.agent_row][self.world.agent_col-1].append('p')
        except IndexError:
            pass

    def predict_wumpus(self):
        try:
            if 'S' in self.world.world[self.world.agent_row][self.world.agent_col]:
                if self.world.agent_row-1 >= 0:
                    if '.' not in self.world.world[self.world.agent_row-1][self.world.agent_col]:
                        if 'w' not in self.world_knowledge[self.world.agent_row-1][self.world.agent_col]:
                            self.world_knowledge[self.world.agent_row-1][self.world.agent_col].append('w')
        except IndexError:
            pass
        try:
            if 'S' in self.world.world[self.world.agent_row][self.world.agent_col]:
                if self.world.agent_col+1 < self.world.num_cols:
                    if '.' not in self.world.world[self.world.agent_row][self.world.agent_col+1]:
                        if 'w' not in self.world_knowledge[self.world.agent_row][self.world.agent_col+1]:
                            self.world_knowledge[self.world.agent_row][self.world.agent_col+1].append('w')
        except IndexError:
            pass
        try:
            if 'S' in self.world.world[self.world.agent_row][self.world.agent_col]:
                if self.world.agent_row+1 < self.world.num_rows:
                    if '.' not in self.world.world[self.world.agent_row+1][self.world.agent_col]:
                        if 'w' not in self.world_knowledge[self.world.agent_row+1][self.world.agent_col]:
                            self.world_knowledge[self.world.agent_row+1][self.world.agent_col].append('w')
        except IndexError:
            pass
        try:
            if 'S' in self.world.world[self.world.agent_row][self.world.agent_col]:
                if self.world.agent_col-1 >= 0:
                    if '.' not in self.world.world[self.world.agent_row][self.world.agent_col-1]:
                        if 'w' not in self.world_knowledge[self.world.agent_row][self.world.agent_col-1]:
                            self.world_knowledge[self.world.agent_row][self.world.agent_col-1].append('w')
        except IndexError:
            pass

    def clean_predictions(self):
        self.num_stenches = 0
        for i in range(self.world.num_rows):
            for j in range(self.world.num_cols):
                if 'S' in self.world_knowledge[i][j]:
                    self.num_stenches += 1
                if 'w' in self.world_knowledge[i][j]:
                    try:
                        if i-1 >= 0:
                            if '.' in self.world_knowledge[i-1][j]:
                                if 'S' not in self.world_knowledge[i-1][j]:
                                    self.world_knowledge[i][j].remove('w')
                                    self.world_knowledge[i][j].append('nw')
                    except IndexError:
                        pass
                    try:
                        if j+1 < self.world.num_cols:
                            if '.' in self.world_knowledge[i][j+1]:
                                if 'S' not in self.world_knowledge[i][j+1]:
                                    self.world_knowledge[i][j].remove('w')
                                    self.world_knowledge[i][j].append('nw')
                    except IndexError:
                        pass
                    try:
                        if i+1 < self.world.num_rows:
                            if '.' in self.world_knowledge[i+1][j]:
                                if 'S' not in self.world_knowledge[i+1][j]:
                                    self.world_knowledge[i][j].remove('w')
                                    self.world_knowledge[i][j].append('nw')
                    except IndexError:
                        pass
                    try:
                        if j-1 >= 0:
                            if '.' in self.world_knowledge[i][j-1]:
                                if 'S' not in self.world_knowledge[i][j-1]:
                                    self.world_knowledge[i][j].remove('w')
                                    self.world_knowledge[i][j].append('nw')
                    except IndexError:
                        pass
                if 'p' in self.world_knowledge[i][j]:
                    try:
                        if i-1 >= 0:
                            if '.' in self.world_knowledge[i-1][j]:
                                if 'B' not in self.world_knowledge[i-1][j]:
                                    self.world_knowledge[i][j].remove('p')
                                    self.world_knowledge[i][j].append('np')
                    except IndexError:
                        pass
                    try:
                        if j+1 < self.world.num_cols:
                            if '.' in self.world_knowledge[i][j+1]:
                                if 'B' not in self.world_knowledge[i][j+1]:
                                    self.world_knowledge[i][j].remove('p')
                                    self.world_knowledge[i][j].append('np')
                    except IndexError:
                        pass
                    try:
                        if i+1 < self.world.num_rows:
                            if '.' in self.world_knowledge[i+1][j]:
                                if 'B' not in self.world_knowledge[i+1][j]:
                                    self.world_knowledge[i][j].remove('p')
                                    self.world_knowledge[i][j].append('np')
                    except IndexError:
                        pass
                    try:
                        if j-1 >= 0:
                            if '.' in self.world_knowledge[i][j-1]:
                                if 'B' not in self.world_knowledge[i][j-1]:
                                    self.world_knowledge[i][j].remove('p')
                                    self.world_knowledge[i][j].append('np')
                    except IndexError:
                        pass

    def confirm_wumpus_knowledge(self):
        for i in range(self.world.num_rows):
            for j in range(self.world.num_cols):
                if 'w' in self.world_knowledge[i][j]:
                    stenches_around = 0
                    try:
                        if i-1 >= 0:
                            if 'S' in self.world_knowledge[i-1][j]:
                                stenches_around += 1
                    except IndexError:
                        pass
                    try:
                        if j+1 < self.world.num_cols:
                            if 'S' in self.world_knowledge[i][j+1]:
                                stenches_around += 1
                    except IndexError:
                        pass
                    try:
                        if i+1 < self.world.num_rows:
                            if 'S' in self.world_knowledge[i+1][j]:
                                stenches_around += 1
                    except IndexError:
                        pass
                    try:
                        if j-1 >= 0:
                            if 'S' in self.world_knowledge[i][j-1]:
                                stenches_around += 1
                    except IndexError:
                        pass
                    if stenches_around < self.num_stenches:
                        self.world_knowledge[i][j].remove('w')
                        self.world_knowledge[i][j].append('nw')

    def move_up(self):
        try:
            if self.world.agent_row-1 >= 0:
                self.remove_agent()
                self.world.agent_row -= 1
                self.add_agent()
                return True
            else:
                return False
        except IndexError:
            return False

    def move_right(self):
        try:
            if self.world.agent_col+1 < self.world.num_cols:
                self.remove_agent()
                self.world.agent_col += 1
                self.add_agent()
                return True
            else:
                return False
        except IndexError:
            return False

    def move_down(self):
        try:
            if self.world.agent_row+1 < self.world.num_rows:
                self.remove_agent()
                self.world.agent_row += 1
                self.add_agent()
                return True
            else:
                return False
        except IndexError:
            return False

    def move_left(self):
        try:
            if self.world.agent_col-1 >= 0:
                self.remove_agent()
                self.world.agent_col -= 1
                self.add_agent()
                return True
            else:
                return False
        except IndexError:
            return False

    def remove_agent(self):
        self.world.world[self.world.agent_row][self.world.agent_col].remove('A')
        self.world_knowledge[self.world.agent_row][self.world.agent_col].remove('A')

    def add_agent(self):
        self.world.world[self.world.agent_row][self.world.agent_col].append('A')
        self.world_knowledge[self.world.agent_row][self.world.agent_col].append('A')

    def mark_tile_visited(self):
        if '.' not in self.world_knowledge[self.world.agent_row][self.world.agent_col]:
            self.world.world[self.world.agent_row][self.world.agent_col].append('.')
            self.world_knowledge[self.world.agent_row][self.world.agent_col].append('.')

    def is_dead(self):
        if 'W' in self.world.world[self.world.agent_row][self.world.agent_col]:
            print("You have been slain by the Wumpus!")
            return True
        elif 'P' in self.world.world[self.world.agent_row][self.world.agent_col]:
            print("You have fallen in a pit!")
            return True
        else:
            return False

    def is_safe_move(self, row, col):
        try:
            if 'w' in self.world_knowledge[row][col]:
                return False
        except IndexError:
            pass
        try:
            if 'p' in self.world_knowledge[row][col]:
                return False
        except IndexError:
            pass
        try:
            if 'W' in self.world_knowledge[row][col]:
                return False
        except IndexError:
            pass
        try:
            if 'P' in self.world_knowledge[row][col]:
                return False
        except IndexError:
            pass
        return True

Grid Label

In [6]:
class Grid_Label():
    def __init__(self, master, i, j):
        self.text = StringVar()
        self.label = Label(master, textvariable = self.text, height = 5, width = 11, relief = RIDGE, bg = "gray30", fg = "white", font = "Helvetica 14 bold")
        self.label.grid(row = i, column = j, sticky = W, pady = 1)
        self.row = i
        self.col = j
    def change_text(self, updated_text):
        self.text.set(str(updated_text))


GUI

In [7]:
def solve_wumpus_world(master, world_file):
    world = World()
    world.generate_world(world_file)
    label_grid = [[Grid_Label(master, i, j) for j in range(world.num_cols)] for i in range(world.num_rows)]
    agent = Agent(world, label_grid)
    # Agent Solving
    while agent.exited == False:
        agent.explore()
        if agent.found_gold == True:
            agent.leave_cave()
        break
    agent.repaint_world()
    agent.world_knowledge[agent.world.agent_row][agent.world.agent_col].remove('A')
    time.sleep(1.5)
    agent.repaint_world()

master = Tk()
master.title("Wumpus World")
world = World()
world.generate_world("world_1.txt")
label_grid = [[Grid_Label(master, i, j) for j in range(world.num_cols)] for i in range(world.num_rows)]
world_1 = Button(master, text="World 1",  command= lambda: solve_wumpus_world(master, "world_1.txt"), width=8, font = "Helvetica 14 bold", bg = "gray80", fg = "gray40", borderwidth=0, activeforeground="white", activebackground="gray40")
world_1.grid(row = 0, column = len(label_grid[0]))
mainloop()

FileNotFoundError: [Errno 2] No such file or directory: 'world_1.txt'